In [2]:
cd C:\Users\Sharvari\Documents\GitHub\VishwaniketanDSWorkshop2016

C:\Users\Sharvari\Documents\GitHub\VishwaniketanDSWorkshop2016


In [3]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

#Print you can execute arbitrary python code
train = pd.read_csv("t.csv", dtype={"Age": np.float64}, )
test = pd.read_csv("test.csv", dtype={"Age": np.float64}, )

#Print to standard output, and see the results in the "log" section below after running your script
print("\n\nTop of the training data:")
print(train.head())

print("\n\nSummary statistics of training data")
print(train.describe())

# Split train set to X(features), Y(labels)
print("\n\nTrain set size: %s" % (train.shape,))
X = train.loc[:, ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Cabin", "Embarked"]]
Y = train.loc[:,"Survived"]
print("\n\nTop of features:")
print(X.head())
print("\n\nTop of labels(1 Survived, 0 Not)")
print(Y.head())

# Data Clean
# For Sex, convert male ->1, female -> 0
sex_map = {"male":1, "female":0}
X.loc[:, "Sex"] = X.loc[:, "Sex"].map(sex_map)

## Fix missing data
print("\n\nFind which columns contains NaN value:")
for i in range(X.columns.size):
    if X.iloc[:,i].isnull().sum() > 0 :
        print("%s :%d Nan" % (X.columns[i], X.iloc[:,i].isnull().sum()))
        
## For age use mean as missing ones
mean_age = int(X.loc[:,"Age"].mean())
print("\n\nUse mean age=%s to fill null value" % (mean_age))
X.loc[:,"Age"] = X.loc[:, "Age"].fillna(mean_age)

## For cabin and embarked use probability generate missing data
## cabin only keep first letter
dict_cabin = {}
for cabin in X.loc[:, "Cabin"]:
    cabin = ord(cabin[0])-ord('A') if pd.notnull(cabin) else cabin 
    if cabin not in dict_cabin:
        dict_cabin[cabin] = 1
    else:
        dict_cabin[cabin] += 1
print("\n\nCabin info:%s" % (dict_cabin))
print("\ncabin lost too many infos, so delete it")
X.pop("Cabin")

dict_embarked = {}
for embarked in X.loc[:, "Embarked"]:
    if embarked not in dict_embarked:
        dict_embarked[embarked] = 1
    else:
        dict_embarked[embarked] += 1
print("\nEmbarked info:%s" % (dict_embarked))
## cause 'S' is the one which appear the most
X.loc[:,"Embarked"] = X.loc[:, "Embarked"].fillna("S")
embarked_map = lambda e: ord(e)-ord("A")
X.loc[:,"Embarked"] = X.loc[:, "Embarked"].map(embarked_map)

# Look at features now
print("\n\nTop of features:")
print(X.head())

# Train classifier
clf = RandomForestClassifier(n_estimators=int(X.shape[1] / 2))
scores = cross_validation.cross_val_score(clf, X, Y, cv=5)
print("RandomForese Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
clf = clf.fit(X, Y)

# Features selecting
print(clf.feature_importances_)

# Process test data
#Print to standard output, and see the results in the "log" section below after running your script
print("\n\nTop of the test data:")
print(test.head())

print("\n\nSummary statistics of test data")
print(test.describe())

to_predict = test.loc[:, ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
## Fix missing data
print("\n\nFind which columns in test contains NaN value:")
for i in range(to_predict.columns.size):
    if to_predict.iloc[:,i].isnull().sum() > 0 :
        print("%s :%d Nan" % (to_predict.columns[i], to_predict.iloc[:,i].isnull().sum()))
to_predict.loc[:,"Age"] = to_predict.loc[:, "Age"].fillna(to_predict.loc[:, "Age"].mean())
to_predict.loc[:,"Fare"] = to_predict.loc[:, "Fare"].fillna(to_predict.loc[:, "Fare"].mean())
to_predict.loc[:,"Embarked"] = to_predict.loc[:, "Embarked"].map(embarked_map)
to_predict.loc[:, "Sex"] = to_predict.loc[:, "Sex"].map(sex_map)

# predict
result = clf.predict(to_predict)
prediction = pd.DataFrame({"PassengerId": test.loc[:,"PassengerId"], "Survived": result})
prediction.to_csv('result.csv', index=False)


C:\Users\Sharvari\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)




Top of the training data:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450 

C:\Users\Sharvari\Anaconda2\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


RandomForese Accuracy: 0.77 (+/- 0.08)
[ 0.10277277  0.27310351  0.24801644  0.04140708  0.03487801  0.26706124
  0.03276095]


Top of the test data:
   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


Summary statistics of tes